In [1]:
import os
import os.path as osp
import sys
from concurrent.futures import ProcessPoolExecutor

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa as rosa
from tqdm import tqdm_notebook as tqdm

from ismir import ISMIR2004

%load_ext autoreload
%autoreload 2

In [2]:
for phase in ('train', 'val', 'test'):
    dataset = ISMIR2004(phase=phase, root='/home/youchen/tmp_datasets/ISMIR2004')
    paths = [(osp.join(dataset.root, 'audio', dataset._phase, x),
              osp.join('/home/youchen/tmp_datasets/ISMIR2004/precomputed/vanilla/{}/{}.npy'.format(dataset._phase, x)))
             for x in dataset.X]
    for input_path, output_path in paths:
        dir_name = osp.dirname(output_path)
        if not osp.exists(dir_name):
            os.makedirs(dir_name)
    
    
    def process_single(paths):
        input_path, output_path = paths
        if osp.exists(output_path):
            return
        try:
            x, _ = rosa.load(input_path)
            x = x.astype(np.float32)
        except Exception:
            print(input_path)
            return
        try:
            with open(output_path, 'wb') as f:
                np.save(f, x)
        except Exception as e:
            print(e)
            if osp.exists(output_path):
                os.remove(output_path)

                
    with ProcessPoolExecutor(20) as executor:
        for processed in tqdm(executor.map(process_single, paths), total=len(paths)):
            pass

In [16]:
%%time
for phase in ('train', 'val', 'test'):
    dataset = ISMIR2004(phase=phase, root='/home/youchen/tmp_datasets/ISMIR2004')
    for idx, (x, y) in tqdm(zip(range(len(dataset)), dataset), total=len(dataset)):
        if len(x) < 22050 * 30:
            print(phase, idx+1)

train 65
train 73
train 77
train 134
train 544


val 69
val 70
val 78
val 80
val 82


test 74
test 76
test 133
CPU times: user 59min 21s, sys: 2min 29s, total: 1h 1min 51s
Wall time: 10min 1s


In [8]:
dataset.X[72]

'classical/the_english_concert_with_trevor_pinnock/handel_tamerlano_act_ii/3-tamerlano_act_i_track_3.mp3'

In [9]:
dataset[72][0].shape

(201048,)